In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from scipy import stats

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

train_data.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price
0,1292132,Land Rover Range Rover Velar 3.0A Si6 R-Dynami...,land rover,range,"1 owner, no repairs needed! it looks great, in...",2018.0,NaN,08-mar-2018,suv,parf car,...,96000.0,88906.0,132031.0,NaN,NaN,uncategorized,3l supercharged v6 p380 engine at 375bhp/450nm...,"2 x massage/memory/cooling & warmer seat, rear...",NaN,193788.0
1,1294696,Mercedes-Benz C-Class C200 Sport Premium Sunroof,mercedes-benz,c200,rare beautiful white c200 sport premium sunroo...,2017.0,NaN,28-dec-2017,luxury sedan,"parf car, premium ad car",...,85680.0,40678.0,43950.0,NaN,NaN,uncategorized,"2.0l 4 cylinders inline turbocharged engine, p...","multi function steering, electric tailgate, re...",NaN,96800.0
2,1311717,Honda Odyssey 2.4A (COE till 09/2027),honda,odyssey,comes with warranty. full service done.,2007.0,NaN,19-sep-2007,mpv,"coe car, premium ad car, low mileage car",...,138000.0,27994.0,30794.0,NaN,NaN,uncategorized,"2.4l k24a 4 cylinders inline dohc i-vtec, 5 sp...","cruise control, touchscreen audio, reverse cam...",NaN,39800.0
3,1310068,Toyota Corolla Altis 1.6A (COE till 12/2028),toyota,altis,0,2008.0,NaN,15-dec-2008,mid-sized sedan,"coe car, premium ad car",...,160000.0,16084.0,16084.0,NaN,NaN,uncategorized,super fuel efficient 1.6l 16 valves dohc vvt-i...,"leather seats, pioneer dvd audio system with r...",NaN,44800.0
4,1325280,Lexus GS300 (COE till 06/2026),lexus,gs,wear and tear done up. well maintained and reg...,2006.0,NaN,22-dec-2006,luxury sedan,"coe car, premium ad car",...,183000.0,50414.0,55456.0,NaN,NaN,uncategorized,"powerful 3.0l v6 engine, 227bhp, 6 speed , key...",premium upholstery electric seats. memory seat...,NaN,25800.0


In [3]:
print('Training data number = {}'.format(train_data.shape[0]))
print('Test data number = {}\n'.format(test_data.shape[0]))
train_data.columns 

Training data number = 25000
Test data number = 10000



Index(['listing_id', 'title', 'make', 'model', 'description', 'manufactured',
       'original_reg_date', 'reg_date', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap',
       'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value',
       'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category',
       'features', 'accessories', 'indicative_price', 'price'],
      dtype='object')

In [4]:
train_data.isna().sum()

listing_id               0
title                    0
make                  1316
model                    0
description            680
manufactured             7
original_reg_date    24745
reg_date                 0
type_of_vehicle          0
category                 0
transmission             0
curb_weight            307
power                 2640
fuel_type            19121
engine_cap             596
no_of_owners            18
depreciation           507
coe                      0
road_tax              2632
dereg_value            220
mileage               5304
omv                     64
arf                    174
opc_scheme           24838
lifespan             22671
eco_category             0
features               843
accessories           3813
indicative_price     25000
price                    0
dtype: int64

In [5]:
# Drop unnecessary columns
train_data.drop(columns=['listing_id', 'original_reg_date', 'category', 'opc_scheme', 'lifespan', 'eco_category', 'indicative_price', 'description'], inplace=True)
test_data.drop(columns=['listing_id', 'original_reg_date', 'category', 'opc_scheme', 'lifespan', 'eco_category', 'indicative_price', 'description'], inplace=True)

# Function to extract COE expiry date and make from 'title'
def extract_features(title):    
    # Extract COE expiry date (e.g., COE till 01/2027)
    coe_info = re.search(r'\(COE till (\d{2}/\d{4})\)', title)
    coe_expiry = coe_info.group(1) if coe_info else None
    
    # Extract make (first word in title)
    make = title.split()[0] if title else None

    return pd.Series([coe_expiry, make])

# Apply the extract_features function to both train and test data
train_data[['coe_expiry', 'extracted_make']] = train_data['title'].apply(extract_features)
test_data[['coe_expiry', 'extracted_make']] = test_data['title'].apply(extract_features)

# Fill missing 'make' values with the extracted make from title
train_data['make'] = train_data['make'].fillna(train_data['extracted_make'])
test_data['make'] = test_data['make'].fillna(test_data['extracted_make'])

# Drop the 'title' and 'extracted_make' columns
train_data.drop(columns=['title', 'extracted_make'], inplace=True)
test_data.drop(columns=['title', 'extracted_make'], inplace=True)

# Function to extract year and month from 'coe_expiry'
def extract_year_month(date_str):
    try:
        month, year = date_str.split('/')
        return int(year), int(month)
    except:
        return None, None  # Handle any unexpected format

# Apply the function to 'coe_expiry' in train_data and test_data
train_data['coe_expiry_year'], train_data['coe_expiry_month'] = zip(*train_data['coe_expiry'].apply(extract_year_month))
test_data['coe_expiry_year'], test_data['coe_expiry_month'] = zip(*test_data['coe_expiry'].apply(extract_year_month))

# drop the original 'coe_expiry' column
train_data.drop(columns=['coe_expiry'], inplace=True)
test_data.drop(columns=['coe_expiry'], inplace=True)

In [6]:
# Impute 'engine_cap' based on 'curb_weight' groups
train_data['engine_cap'] = train_data.groupby(['curb_weight'])['engine_cap'].transform(lambda x: x.fillna(x.mean()))
test_data['engine_cap'] = test_data.groupby(['curb_weight'])['engine_cap'].transform(lambda x: x.fillna(x.mean()))

# Use KNN imputer for 'curb_weight' and 'engine_cap'
knn_imputer = KNNImputer(n_neighbors=5)
num_cols = ['curb_weight', 'engine_cap']

train_data[num_cols] = knn_imputer.fit_transform(train_data[num_cols])
test_data[num_cols] = knn_imputer.transform(test_data[num_cols])

# Check for missing values after imputation
print('Missing values after imputing engine_cap and curb_weight:\n', train_data.isna().sum())

Missing values after imputing engine_cap and curb_weight:
 make                    0
model                   0
manufactured            7
reg_date                0
type_of_vehicle         0
transmission            0
curb_weight             0
power                2640
fuel_type           19121
engine_cap              0
no_of_owners           18
depreciation          507
coe                     0
road_tax             2632
dereg_value           220
mileage              5304
omv                    64
arf                   174
features              843
accessories          3813
price                   0
coe_expiry_year     19032
coe_expiry_month    19032
dtype: int64


In [7]:
# Convert 'reg_date' to datetime format
train_data['reg_date'] = pd.to_datetime(train_data['reg_date'], format='%d-%b-%Y')
test_data['reg_date'] = pd.to_datetime(test_data['reg_date'], format='%d-%b-%Y')

# Extract 'reg_year', 'reg_month', and 'reg_day' from 'reg_date'
train_data['reg_year'] = train_data['reg_date'].dt.year
train_data['reg_month'] = train_data['reg_date'].dt.month
train_data['reg_day'] = train_data['reg_date'].dt.day

test_data['reg_year'] = test_data['reg_date'].dt.year
test_data['reg_month'] = test_data['reg_date'].dt.month
test_data['reg_day'] = test_data['reg_date'].dt.day

# Drop the original 'reg_date' column
train_data.drop(columns=['reg_date'], inplace=True)
test_data.drop(columns=['reg_date'], inplace=True)

# Fill missing 'manufactured' values with 'reg_year'
train_data['manufactured'].fillna(train_data['reg_year'], inplace=True)
test_data['manufactured'].fillna(test_data['reg_year'], inplace=True)

# Create 'car_age' feature
current_year = 2024
train_data['car_age'] = current_year - train_data['manufactured']
test_data['car_age'] = current_year - test_data['manufactured']

# Function to fill 'no_of_owners' based on 'manufactured' year
def fill_no_of_owners_based_on_manufactured(df):
    # Group by 'manufactured' and fill 'no_of_owners' based on the mode within each group
    return df['no_of_owners'].fillna(df.groupby('manufactured')['no_of_owners'].transform(lambda x: x.mode()[0] if not x.mode().empty else x.median()))

# Apply this function to both train_data and test_data
train_data['no_of_owners'] = fill_no_of_owners_based_on_manufactured(train_data)
test_data['no_of_owners'] = fill_no_of_owners_based_on_manufactured(test_data)

# Fill missing 'coe_expiry_year' with 'reg_year' + 20
train_data['coe_expiry_year'] = train_data['coe_expiry_year'].fillna(train_data['reg_year'] + 20)
test_data['coe_expiry_year'] = test_data['coe_expiry_year'].fillna(test_data['reg_year'] + 20)

# Fill missing 'coe_expiry_month' with 'reg_month'
train_data['coe_expiry_month'] = train_data['coe_expiry_month'].fillna(train_data['reg_month'])
test_data['coe_expiry_month'] = test_data['coe_expiry_month'].fillna(test_data['reg_month'])

# Verify missing values
print('Missing values after imputing COE expiry:\n', train_data.isna().sum())

Missing values after imputing COE expiry:
 make                    0
model                   0
manufactured            0
type_of_vehicle         0
transmission            0
curb_weight             0
power                2640
fuel_type           19121
engine_cap              0
no_of_owners            0
depreciation          507
coe                     0
road_tax             2632
dereg_value           220
mileage              5304
omv                    64
arf                   174
features              843
accessories          3813
price                   0
coe_expiry_year         0
coe_expiry_month        0
reg_year                0
reg_month               0
reg_day                 0
car_age                 0
dtype: int64


In [8]:
# Create indicators for missing values in certain columns
for col in ['power', 'curb_weight', 'mileage']:
    train_data[col + '_missing'] = train_data[col].isna().astype(int)
    test_data[col + '_missing'] = test_data[col].isna().astype(int)

# Group-based imputation for 'mileage' based on 'type_of_vehicle' and 'manufactured'
group_cols = ['type_of_vehicle', 'manufactured']

train_data['mileage'] = train_data.groupby(group_cols)['mileage'].transform(lambda x: x.fillna(x.mean()))
test_data['mileage'] = test_data.groupby(group_cols)['mileage'].transform(lambda x: x.fillna(x.mean()))

# Linear regression imputation for remaining missing 'mileage'
train_non_missing = train_data.dropna(subset=['mileage', 'manufactured'])
X_train = train_non_missing[['manufactured']].astype(float)  
y_train = train_non_missing['mileage'].astype(float)

model = LinearRegression()
model.fit(X_train, y_train)

# Predict missing 'mileage' in training data
train_missing_rt = train_data[(train_data['mileage'].isna()) & (train_data['manufactured'].notna())]
train_data.loc[train_missing_rt.index, 'mileage'] = model.predict(train_missing_rt[['manufactured']].astype(float))

# Predict missing 'mileage' in test data
test_missing_rt = test_data[(test_data['mileage'].isna()) & (test_data['manufactured'].notna())]
test_data.loc[test_missing_rt.index, 'mileage'] = model.predict(test_missing_rt[['manufactured']].astype(float))

# Impute 'road_tax' based on 'engine_cap' groups
train_data['road_tax'] = train_data.groupby('engine_cap')['road_tax'].transform(lambda x: x.fillna(x.mean()))
test_data['road_tax'] = test_data.groupby('engine_cap')['road_tax'].transform(lambda x: x.fillna(x.mean()))

In [9]:
# Linear regression imputation for remaining missing 'road_tax'
train_non_missing = train_data.dropna(subset=['engine_cap', 'road_tax'])
X_train = train_non_missing[['engine_cap']].astype(float) 
y_train = train_non_missing['road_tax'].astype(float)

model = LinearRegression()
model.fit(X_train, y_train)

# Predict missing 'road_tax' in training data
train_missing_rt = train_data[(train_data['road_tax'].isna()) & (train_data['engine_cap'].notna())]
train_data.loc[train_missing_rt.index, 'road_tax'] = model.predict(train_missing_rt[['engine_cap']].astype(float))

# Predict missing 'road_tax' in test data
test_missing_rt = test_data[(test_data['road_tax'].isna()) & (test_data['engine_cap'].notna())]
test_data.loc[test_missing_rt.index, 'road_tax'] = model.predict(test_missing_rt[['engine_cap']].astype(float))

# Verify missing values
print('Missing values after imputing road_tax:\n', train_data.isna().sum())

Missing values after imputing road_tax:
 make                       0
model                      0
manufactured               0
type_of_vehicle            0
transmission               0
curb_weight                0
power                   2640
fuel_type              19121
engine_cap                 0
no_of_owners               0
depreciation             507
coe                        0
road_tax                   0
dereg_value              220
mileage                    0
omv                       64
arf                      174
features                 843
accessories             3813
price                      0
coe_expiry_year            0
coe_expiry_month           0
reg_year                   0
reg_month                  0
reg_day                    0
car_age                    0
power_missing              0
curb_weight_missing        0
mileage_missing            0
dtype: int64


In [10]:
# Function to impute 'power' using LGBMRegressor
def impute_with_model(train_data, target_col, feature_cols):
    # Impute missing values in feature columns using KNNImputer
    knn_imputer = KNNImputer(n_neighbors=5)
    train_data[feature_cols] = knn_imputer.fit_transform(train_data[feature_cols])
    
    model = LGBMRegressor()

     # Split into known and unknown target values
    known = train_data[train_data[target_col].notna()]
    unknown = train_data[train_data[target_col].isna()]

    # Train the model
    model.fit(known[feature_cols], known[target_col])

    # Predict missing values
    train_data.loc[train_data[target_col].isna(), target_col] = model.predict(unknown[feature_cols])
    
    return train_data

# Define features to use for imputation
feature_cols = ['curb_weight', 'engine_cap', 'road_tax', 'omv']

# Impute 'power' in both datasets
train_data = impute_with_model(train_data, 'power', feature_cols)
test_data = impute_with_model(test_data, 'power', feature_cols)

# Create 'power_to_weight' ratio
train_data['power_to_weight'] = train_data['power'] / train_data['curb_weight']
test_data['power_to_weight'] = test_data['power'] / test_data['curb_weight']

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000276 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 941
[LightGBM] [Info] Number of data points in the train set: 22360, number of used features: 4
[LightGBM] [Info] Start training from score 140.908095
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000117 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 931
[LightGBM] [Info] Number of data points in the train set: 8914, number of used features: 4
[LightGBM] [Info] Start training from score 140.861230


In [11]:
# Impute continuous features using KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
num_cols = ['depreciation', 'dereg_value', 'omv', 'arf']

train_data[num_cols] = knn_imputer.fit_transform(train_data[num_cols])
test_data[num_cols] = knn_imputer.transform(test_data[num_cols])

In [12]:
# Fill 'fuel_type' with 'diesel' for cars manufactured in or before 2005
train_data.loc[train_data['manufactured'] <= 2005, 'fuel_type'] = train_data.loc[train_data['manufactured'] <= 2005, 'fuel_type'].fillna('diesel')
test_data.loc[test_data['manufactured'] <= 2005, 'fuel_type'] = test_data.loc[test_data['manufactured'] <= 2005, 'fuel_type'].fillna('diesel')

# Fill 'fuel_type' with 'electric' for specific makes
electric_makes = ['aion', 'byd', 'tesla', 'dfsk', 'kyc', 'mg', 'polestar', 'shineray', 'smart', 'sokon']
train_data.loc[train_data['make'].isin(electric_makes), 'fuel_type'] = train_data.loc[train_data['make'].isin(electric_makes), 'fuel_type'].fillna('electric')
test_data.loc[test_data['make'].isin(electric_makes), 'fuel_type'] = test_data.loc[test_data['make'].isin(electric_makes), 'fuel_type'].fillna('electric')

# Fill remaining 'fuel_type' missing values based on mode within 'make'
def fill_fuel_type_by_make(df):
    for make in df['make'].unique():
        make_mode = df.loc[df['make'] == make, 'fuel_type'].mode()
        if not make_mode.empty:
            # Fill by mode of the 'fuel_type' for that make
            df.loc[df['make'] == make, 'fuel_type'] = df.loc[df['make'] == make, 'fuel_type'].fillna(make_mode[0])
        else:
            # If all 'fuel_type' are NaN for this make, use overall mode
            overall_mode = df['fuel_type'].mode()[0]
            df.loc[df['make'] == make, 'fuel_type'] = df.loc[df['make'] == make, 'fuel_type'].fillna(overall_mode)
    return df

# Apply the function to both datasets
train_data = fill_fuel_type_by_make(train_data)
test_data = fill_fuel_type_by_make(test_data)

In [13]:
# List of categorical columns
cat_cols = ['make', 'model', 'transmission', 'fuel_type', 'type_of_vehicle']

# Fill missing values with mode
for col in cat_cols:
    train_data[col].fillna(train_data[col].mode()[0], inplace=True)
    test_data[col].fillna(test_data[col].mode()[0], inplace=True)

In [14]:
# One-Hot Encoding for categorical variables
onehot_enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

# Fit encoder on training data and transform both datasets
train_encoded = onehot_enc.fit_transform(train_data[cat_cols])
test_encoded = onehot_enc.transform(test_data[cat_cols])

# Convert encoded arrays to DataFrames
train_encoded_df = pd.DataFrame(train_encoded, columns=onehot_enc.get_feature_names_out(cat_cols))
test_encoded_df = pd.DataFrame(test_encoded, columns=onehot_enc.get_feature_names_out(cat_cols))

# Merge encoded features back to original datasets and drop original categorical columns
train_data = pd.concat([train_data.reset_index(drop=True), train_encoded_df], axis=1).drop(columns=cat_cols)
test_data = pd.concat([test_data.reset_index(drop=True), test_encoded_df], axis=1).drop(columns=cat_cols)

# Drop 'features' and 'accessories' columns
train_data.drop(columns=['features', 'accessories'], inplace=True)
test_data.drop(columns=['features', 'accessories'], inplace=True)

In [15]:
# Check for any remaining missing values in training data
print("Columns with missing values in train_data:")
print(train_data.isna().sum()[train_data.isna().sum() > 0])

# Check for any remaining missing values in test data
print("Columns with missing values in test_data:")
print(test_data.isna().sum()[test_data.isna().sum() > 0])

Columns with missing values in train_data:
Series([], dtype: int64)
Columns with missing values in test_data:
Series([], dtype: int64)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(columns = ['price']), train_data['price'], test_size=0.2, random_state=42)

X_train
X_test
y_train
y_test

6868      31777.0
24016     53800.0
9668     129000.0
13640     45800.0
14018     58900.0
           ...   
8670     146800.0
11839     23777.0
4013     150777.0
21147     57888.0
695       99800.0
Name: price, Length: 5000, dtype: float64

In [17]:
# Check for missing values in the splits
print("NaNs in X_train:", X_train.isna().sum().sum())
print("NaNs in X_test:", X_test.isna().sum().sum())

NaNs in X_train: 0
NaNs in X_test: 0


In [18]:
# Train Bagging Regressor
model1 = BaggingRegressor(random_state=50, max_samples = 950)
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

In [19]:
# Train Random Forest Regressor
model2 = RandomForestRegressor(random_state=50,  max_features=0.43, min_samples_leaf=1,)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

In [20]:
# Train Gradient Boosting Regressor
model3 = GradientBoostingRegressor(
    random_state=50, 
    min_samples_split = 6, 
    min_samples_leaf = 3, 
    max_depth = 7
)

model3.fit(X_train, y_train) 

y_pred3 = model3.predict(X_test)

In [21]:
# Residual fitting with Gradient Boosting on Random Forest residuals
training_residuals = y_train - model2.predict(X_train)
model3.fit(X_train, training_residuals)
pred_residuals = model3.predict(X_test)
y_pred4 = pred_residuals + model2.predict(X_test)

In [22]:
y_pred = [y_pred1, y_pred2, y_pred3, y_pred4]

In [23]:
def rmse_score(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [24]:
# Evaluate each model
method = 1
for test in y_pred:
    r2_test = r2_score(y_test, test)
    rmse = rmse_score(y_test, test)
    print(f"R-squared (Test): {r2_test}")
    print(f"RMSE{method}: {rmse}\n")
    method+=1 

R-squared (Test): 0.9109065478592873
RMSE1: 42840.07944500033

R-squared (Test): 0.9701329184768154
RMSE2: 24804.116798632345

R-squared (Test): 0.9562791521199849
RMSE3: 30010.391555116094

R-squared (Test): 0.9708944619647982
RMSE4: 24485.850298347832



In [25]:
# Residual fitting with Gradient Boosting on Random Forest residuals
training_residuals = y_train - model2.predict(X_train)
model3.fit(X_train, training_residuals)
pred_residuals = model3.predict(test_data)
final_predictions = pred_residuals + model2.predict(test_data)

# Prepare the output DataFrame
output_df = pd.DataFrame({
    'Id': range(len(final_predictions)), 
    'Predicted': final_predictions        
})

# Round predictions to 1 decimal place
output_df['Predicted'] = output_df['Predicted'].round(1)

# Export the predictions to a CSV file
output_df.to_csv('submission.csv', index=False)
print("Done")

Done
